In [1]:
#grab dependencies
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
from functools import partial,reduce
import seaborn as sns
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [2]:
#get path
path = os.getcwd()
#get list of files
county_files = os.listdir(path)
#get only county files
county_files = [file for file in county_files if file.startswith("County")]

#functions
#write a function that will only take in string that contain numbers
def contains_number(s):
    return any(i.isdigit() for i in s)

#grab only state and region name, not all files will have state ane regi specified
def contains_stat_reg(s):
    condition = ['State','StateName','Region','RegionName']
    return(s in condition)

#lets make a loop to convert each csv to a gathered dataframe
list_gathered_data_frames = []
for file in county_files:
    #read in the dataframe
    frame = pd.read_csv(file,encoding = "ISO-8859-1")
    #get list of column names
    col_names = frame.columns
    #get names of columnes to be gathered
    gathered_columns = [column for column in col_names if contains_number(column) == True]
    #identify index columns
    index_columns = [column for column in col_names if contains_number(column) == False]
    #grab only state and region name, not all files will have state ane regi specified
    index_columns_stat_regi = [column for column in index_columns if contains_stat_reg(column) == True]
    #melt the dataframe
    melted_frame = pd.melt(frame=frame, value_vars= gathered_columns, id_vars= index_columns_stat_regi,
                        var_name="Year_Mon",value_name=file)
    #grab column names from melted frame
    melted_frame_columns = melted_frame.columns
    #convert 'StateName' to "State"
    if "StateName" in melted_frame_columns:
        #rename "StateName to "State":
        melted_frame.rename(columns = {"StateName": "State"},inplace=True)
    #convert state abbreviation to statename
    if len(melted_frame['State'][0]) == 2:
        melted_frame["State"] = melted_frame['State'].map(states)
    #add data frame to list
    list_gathered_data_frames.append(melted_frame)
    
#final merge
merged_frame=list_gathered_data_frames[0]
for i in range(len(list_gathered_data_frames)-1):
    merged_frame = pd.merge(merged_frame,list_gathered_data_frames[i+1],how="outer",
                                 on=["RegionName", "Year_Mon","State"])
    #adjust column names
col_names = merged_frame.columns
new_column_names = []
for name in col_names:
    #remove County_
    if "County_" in name:
        name = name.replace("County_","")
    #remove .csv
    if ".csv" in name:
        name = name.replace(".csv","")
    #add back in new_column_names
    new_column_names.append(name)

merged_frame.columns = new_column_names

In [16]:
sum(merged_frame.isnull().values)

array([    0,     0,     0, 11969, 26205, 73616, 12107])

In [5]:
merged_frame.to_csv("Complete_Rental_Values_County.csv")

In [7]:
merged_frame.head()

,RegionName,State,Year_Mon,Zri_SingleFamilyResidenceRental,ZriPerSqft_AllHomes,Zri_MultiFamilyResidenceRental,Zri_AllHomesPlusMultifamily
0,Los Angeles County,California,2010-11,2176.0,1.508,1985.0,2123.0
1,Cook County,Illinois,2010-11,1599.0,1.192,1550.0,1585.0
2,Harris County,Texas,2010-11,1300.0,0.732,1080.0,1295.0
3,Maricopa County,Arizona,2010-11,1247.0,0.752,994.0,1200.0
4,San Diego County,California,2010-11,2192.0,1.364,1799.0,2071.0
